In [0]:
import os

In [0]:
csv_files = [f for f in os.listdir('.') if os.path.isfile(f) and f.split('.')[-1] == 'csv']

In [0]:
import numpy

In [0]:
left_corner_processor = lambda rad: lambda x: x[1]*(1+rad*2)/(x[0]+1+rad) if x[0] < rad else x[1]

In [0]:
def movingaverage(data_list, window_radius:int):
  """
    0 > window_radius > len(data_list)/2
  """
  window_size = 1 + window_radius*2
  if window_size > len(data_list):
    print("ERROR: window is larger than the data")
    return data_list
  if window_radius == 0:
    return data_list
  window= numpy.ones(int(window_size))/float(window_size)
  convolved = list(numpy.convolve(data_list, window, 'full'))[window_radius:-window_radius]
  convolved = list(map(left_corner_processor(window_radius), enumerate(convolved)))
  convolved.reverse()
  convolved = list(map(left_corner_processor(window_radius), enumerate(convolved)))
  convolved.reverse()
  return convolved

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df_filename_s = [(pd.read_csv(f), f) for f in csv_files]

In [0]:
import re

In [0]:
get_name = lambda f: re.split(' - |\.', f)[1]

In [0]:
smoothness = 1

In [0]:
for df_filename in df_filename_s:
  df_filename[0]['sphere_avg'] = movingaverage(df_filename[0]['sphere'], smoothness)
  df_filename[0]['ball_avg'] = movingaverage(df_filename[0]['ball'], smoothness)
  df_filename[0]['ball_dev'] = list(map(lambda x: numpy.abs(x[0]-x[1]), zip(df_filename[0]['ball_avg'], df_filename[0]['ball'])))
  df_filename[0]['sphere_dev'] = list(map(lambda x: numpy.abs(x[0]-x[1]), zip(df_filename[0]['sphere_avg'], df_filename[0]['sphere'])))
  
  plt.plot('R', 'sphere_avg', data=df_filename[0], label='min LMI_P_dB', color='blue', linestyle='dashed')
  plt.plot('R', 'ball_avg', data=df_filename[0], label='min LMI_P', color='red')
  plt.fill_between(df_filename[0]['R'], numpy.subtract(df_filename[0]['ball_avg'], df_filename[0]['ball_dev']),
                                        numpy.add(df_filename[0]['ball_avg'], df_filename[0]['ball_dev']), color='red', alpha=0.1)
  plt.fill_between(df_filename[0]['R'], numpy.subtract(df_filename[0]['sphere_avg'], df_filename[0]['sphere_dev']),
                                        numpy.add(df_filename[0]['sphere_avg'], df_filename[0]['sphere_dev']), color='blue', alpha=0.1)
  
  plt.legend()
  plt.grid()
  plt.xlabel("R")
  plt.ylabel("min value")
#   plt.title(get_name(df_filename[1]))
  plt.savefig(get_name(df_filename[1]), dpi=600)
  plt.clf()
  # plt.show()

<Figure size 432x288 with 0 Axes>